# Censo pecuario ICA 2022

## V. 0.3

In [1]:
# V. 0.3

import os
import pandas as pd
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display, HTML, Markdown
import io

# Directorio que contiene los archivos CSV
directory = "./"

# Obtener la lista de archivos CSV en el directorio
csv_files = [file for file in os.listdir(directory) if file.endswith(".csv")]

# Verificar si hay archivos CSV en el directorio
if len(csv_files) == 0:
    print("No se encontraron archivos CSV en el directorio especificado.")
    exit()

# Cargar los datos de los archivos CSV en un DataFrame
dfs = []

for file in csv_files:
    file_path = os.path.join(directory, file)
    try:
        df = pd.read_csv(file_path)
        dfs.append(df)
    except pd.errors.EmptyDataError:
        print(f"El archivo {file} está vacío y no se puede cargar.")

# Verificar si se cargaron datos en el DataFrame
if len(dfs) == 0:
    print("No se pudo cargar ningún archivo CSV con datos.")
    exit()

# Concatenar los DataFrames en uno solo
data = pd.concat(dfs)

# Mostrar los campos disponibles
fields = data.columns.tolist()
print("Campos disponibles:")
print(fields)

# Crear los dropdown menus para seleccionar los campos
index_dropdown = widgets.Dropdown(options=fields, description="Index:")
variable_dropdown = widgets.Dropdown(options=fields, description="Variable:")
x_axis_dropdown = widgets.Dropdown(options=fields, description="Eje X:")
y_axis_dropdown = widgets.Dropdown(options=fields, description="Eje Y:")
chart_type_dropdown = widgets.Dropdown(options=["bar", "line", "scatter"], description="Tipo de gráfico:")

# Filtrar el campo "DEPARTAMENTO" para obtener las opciones del dropdown de MUNICIPIO
departamento_options = data["DEPARTAMENTO"].unique()

# Crear el dropdown menu para seleccionar el campo "DEPARTAMENTO"
departamento_dropdown = widgets.Dropdown(options=departamento_options, description="Departamento:")

# Crear el dropdown menu para seleccionar el campo "MUNICIPIO" basado en el filtro del campo "DEPARTAMENTO"
municipio_dropdown = widgets.SelectMultiple(description="Municipio:")

# Función para actualizar las opciones del dropdown de MUNICIPIO basado en el filtro del dropdown de DEPARTAMENTO
def update_municipio_options(change):
    selected_departamentos = [departamento_dropdown.value]
    municipio_options = data[data["DEPARTAMENTO"].isin(selected_departamentos)]["MUNICIPIO"].unique()
    municipio_dropdown.options = municipio_options

# Asignar la función de actualización de opciones al evento "value" del dropdown de DEPARTAMENTO
departamento_dropdown.observe(update_municipio_options, 'value')

# Crear las salidas HTML para la tabla y el gráfico
table_output = widgets.Output()
chart_output = widgets.Output()

# Función para generar y mostrar la tabla y el gráfico seleccionados
def generate_output(change):
    # Obtener las selecciones realizadas
    selected_departamentos = [departamento_dropdown.value]
    selected_municipios = list(municipio_dropdown.value)
    
    # Obtener los campos seleccionados
    index_field = index_dropdown.value
    variable_field = variable_dropdown.value
    x_field = x_axis_dropdown.value
    y_field = y_axis_dropdown.value
    chart_type = chart_type_dropdown.value
    
    # Filtrar los datos basado en las selecciones realizadas
    filtered_data = data[data["DEPARTAMENTO"].isin(selected_departamentos) & data["MUNICIPIO"].isin(selected_municipios)]
    
    # Limpiar las salidas HTML
    table_output.clear_output()
    chart_output.clear_output()
    
    # Generar y mostrar la tabla
    with table_output:
        display(HTML("<h2>Tabla</h2>"))
        display(filtered_data)
        
        # Generar el contenido de la tabla en formato markdown
        markdown_content = filtered_data.to_markdown()
        
        # Crear el widget de Textarea para mostrar el contenido en formato markdown
        textarea_widget = widgets.Textarea(value=markdown_content, layout={'height': '100%', 'width': '100%'})
        
        # Crear la función para crear un enlace de descarga
        def create_download_link(content, filename):
            buffer = io.BytesIO(content.encode())
            download_link = widgets.Button(description="Descargar", layout={'width': 'auto'})
            download_link.style.button_color = 'lightgreen'
            download_link.style.font_weight = 'bold'

            def download_table(_):
                with open(filename, 'wb') as f:
                    f.write(buffer.getbuffer())

            download_link.on_click(download_table)
            return download_link
        
        # Mostrar el widget de Textarea
        display(textarea_widget)
        
        # Crear y mostrar el enlace de descarga
        download_link = create_download_link(markdown_content, filename="tabla.md")
        display(download_link)
    
    # Generar y mostrar el gráfico
    with chart_output:
        try:
            fig = go.Figure()
            
            if chart_type == "bar":
                fig.add_trace(go.Bar(
                    x=filtered_data[x_field],
                    y=filtered_data[y_field],
                    name=f"{y_field} ({x_field})"
                ))
            elif chart_type == "line":
                fig.add_trace(go.Scatter(
                    x=filtered_data[x_field],
                    y=filtered_data[y_field],
                    mode="lines",
                    name=f"{y_field} ({x_field})"
                ))
            elif chart_type == "scatter":
                fig.add_trace(go.Scatter(
                    x=filtered_data[x_field],
                    y=filtered_data[y_field],
                    mode="markers",
                    name=f"{y_field} ({x_field})"
                ))

            fig.update_layout(
                title=f"Gráfico de {chart_type} de {y_field} por {x_field}",
                xaxis=dict(title=x_field),
                yaxis=dict(title=y_field)
            )

            fig.show()
        except ValueError as e:
            print(f"No se pudo generar la gráfica. Error: {str(e)}")

# Asignar la función de generación de tabla y gráfico al evento "change" de los dropdown menus
index_dropdown.observe(generate_output, 'value')
variable_dropdown.observe(generate_output, 'value')
x_axis_dropdown.observe(generate_output, 'value')
y_axis_dropdown.observe(generate_output, 'value')
chart_type_dropdown.observe(generate_output, 'value')
departamento_dropdown.observe(generate_output, 'value')
municipio_dropdown.observe(generate_output, 'value')

# Mostrar los dropdown menus y las salidas HTML
display(index_dropdown, variable_dropdown, x_axis_dropdown, y_axis_dropdown, chart_type_dropdown)
display(departamento_dropdown, municipio_dropdown)
display(table_output)
display(chart_output)


Campos disponibles:
['DEPARTAMENTO', 'MUNICIPIO', 'CODIGO MUNICIPIO', 'TERNERAS < 1 AÑO', 'TERNEROS < 1 AÑO', 'HEMBRAS 1 - 2 AÑOS', 'MACHOS 1 - 2 AÑOS', 'HEMBRAS 2 - 3 AÑOS', 'MACHOS 2 - 3 AÑOS', 'HEMBRAS > 3 AÑOS', 'MACHOS > 3 AÑOS', 'TOTAL BOVINOS', ' No DE FINCAS 1 A 50', ' No DE FINCAS 51 A 100', ' No DE FINCAS 101 A 500', ' No DE FINCAS 501 O MAS', 'TOTAL FINCAS CON BOVINOS']


Dropdown(description='Index:', options=('DEPARTAMENTO', 'MUNICIPIO', 'CODIGO MUNICIPIO', 'TERNERAS < 1 AÑO', '…

Dropdown(description='Variable:', options=('DEPARTAMENTO', 'MUNICIPIO', 'CODIGO MUNICIPIO', 'TERNERAS < 1 AÑO'…

Dropdown(description='Eje X:', options=('DEPARTAMENTO', 'MUNICIPIO', 'CODIGO MUNICIPIO', 'TERNERAS < 1 AÑO', '…

Dropdown(description='Eje Y:', options=('DEPARTAMENTO', 'MUNICIPIO', 'CODIGO MUNICIPIO', 'TERNERAS < 1 AÑO', '…

Dropdown(description='Tipo de gráfico:', options=('bar', 'line', 'scatter'), value='bar')

Dropdown(description='Departamento:', options=('ANTIOQUIA', 'ATLANTICO', 'BOGOTA D.C.', 'BOLIVAR', 'BOYACA', '…

SelectMultiple(description='Municipio:', options=(), value=())

Output()

Output()